In [ ]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GroupKFold

## Custom libraries
import index_helpers as ih
from importlib import reload
import data_transformations as dt

In [ ]:
reload(dt)
reload(ih)
## Import, index, and split

segmentation = True
fine_segmentation = True
exclude_expert = False
exclude_meta_data= False

normalization=False
power=False


df = ih.read_and_merge_data(segmentation, fine_segmentation, exclude_expert, exclude_meta_data=False)
df = dt.transformation_call(df, normalization, power)

if segmentation:
  df = ih.index_df_by_person(df)
else:
  df = df.set_index(['File_Name'])
df = ih.categorical_float_to_int(df) #d'ailleurs elle ne sert à rien cette fonction
df = ih.categorical_to_dummy(df, include_expert_as_dummies=False, exclude_meta_data=False)
df1, df2, df3 = ih.separate_expert(df)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score


aucs = []
for df_ in df1, df2, df3, df:
    if segmentation:
    X_train, X_val, y_train, y_val = ih.train_test_split_on_index(features = df_.drop("Label", axis=1),
                                                             label = df_["Label"])
    groups = y_train.reset_index()['File_Name_split']
    else:
    from sklearn.model_selection import train_test_split
    X_train, X_val, y_train, y_val = train_test_split(df1.drop("Label", axis=1), df1["Label"], test_size=0.2)
    groups = y_train.reset_index()['File_Name']

    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)


    ## Make pipeline - name classifier "clf"
    clf_pipeline = Pipeline([("st_scaler", StandardScaler()),
                          ("clf", KNeighborsClassifier())])

    ## Use "clf__" in order to correctly assign parameters to the clf object
    clf_param_grid = {'clf__n_neighbors': list(range(1,30)),
                    'clf__leaf_size': list(range(1,50)),
                    'clf__p': [1, 2]}


    ## Instantiate GroupKFold to avoid data leakage - to be passed to cv
    gkf=GroupKFold(n_splits=2)

    ## Set up Randomized search CV --> modulate n_iter for "quicker" results
    clf_rand_auc = RandomizedSearchCV(estimator=clf_pipeline,
                                    param_distributions=clf_param_grid,
                                    cv=gkf, scoring='roc_auc', verbose=1, n_jobs=2, n_iter=50)

    ## Perform Group K-Cross-validation
    clf_rand_auc.fit(X_train, y_train, groups=groups)

    ## Prediction
    pred = clf_rand_auc.predict(X_val)
    auc = roc_auc_score(y_val, pred)
  
    ## AUC
    aucs.append(auc)

Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    3.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Done  88 tasks      | elapsed:    4.9s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    5.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Done  88 tasks      | elapsed:    5.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    5.8s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   18.7s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   35.6s finished
